In [1]:
import pandas
from sklearn.linear_model import Ridge
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from scipy.sparse import hstack

Загрузите данные об описаниях вакансий и соответствующих годовых зарплатах из файла salary-train.csv

In [2]:
data=pandas.read_csv("salary-train.csv")
data.head(3)

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000


Проведите предобработку:
* Приведите тексты к нижнему регистру (text.lower()).


In [3]:
data['FullDescription']=data['FullDescription'].map(lambda text: text.lower())

* Замените все, кроме букв и цифр, на пробелы — это облегчит дальнейшее разделение текста на слова. Для такой замены в строке text подходит следующий вызов: re.sub('[^a-zA-Z0-9]', ' ', text). Также можно воспользоваться методом replace у DataFrame, чтобы сразу преобразовать все тексты.
```python
train['FullDescription'] = train['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)```


In [4]:
data['FullDescription']=data['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)

* Примените TfidfVectorizer для преобразования текстов в векторы признаков. Оставьте только те слова, которые встречаются хотя бы в 5 объектах (параметр min_df у TfidfVectorizer).

In [5]:
vectorizer=TfidfVectorizer(min_df=5)
X_train=vectorizer.fit_transform(data['FullDescription'])

* Замените пропуски в столбцах LocationNormalized и ContractTime на специальную строку 'nan'. Код для этого был приведен выше.


In [6]:
data['LocationNormalized'].fillna('nan', inplace=True)
data['ContractTime'].fillna('nan', inplace=True)

* Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime.

In [7]:
enc=DictVectorizer()
X_train_enc=enc.fit_transform(data[['LocationNormalized', 'ContractTime']].to_dict('records'))

* Объедините все полученные признаки в одну матрицу "объекты-признаки". Обратите внимание, что матрицы для текстов и категориальных признаков являются разреженными. Для объединения их столбцов нужно воспользоваться функцией scipy.sparse.hstack.

In [8]:
X_train=hstack([X_train, X_train_enc])

Обучите гребневую регрессию с параметрами alpha=1 и random_state=241. Целевая переменная записана в столбце SalaryNormalized.

In [9]:
y_train = data['SalaryNormalized']
model = Ridge(alpha=1, random_state = 241)
model.fit(X_train, y_train)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=241, solver='auto', tol=0.001)

Постройте прогнозы для двух примеров из файла salary-test-mini.csv. Значения полученных прогнозов являются ответом на задание. Укажите их через пробел.

In [10]:
data_pr=pandas.read_csv("salary-test-mini.csv")
data_pr['FullDescription']=data_pr['FullDescription'].map(lambda text: text.lower())
data_pr['FullDescription']=data_pr['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex=True)
data_pr['LocationNormalized'].fillna('nan', inplace=True)
data_pr['ContractTime'].fillna('nan', inplace=True)
data_pr.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,we currently have a vacancy for an hr project ...,Milton Keynes,contract,NaN
1,a web developer opportunity has arisen with an...,Manchester,permanent,NaN


In [11]:
X_test=vectorizer.transform(data_pr['FullDescription'])
X_test_enc=enc.transform(data_pr[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test=hstack([X_test, X_test_enc])

In [12]:
y_test = model.predict(X_test)

In [13]:
print(y_test)

[56555.61500155 37188.32442618]
